# 18.1 线程与协程的对比

演示两个显示动画的程序，一个由 threading 模块使用线程实现，一个借由 asyncio 包使用协程实现。

In [ ]:
# spinner_thread.py

# credits: Adapted from Michele Simionato's
# multiprocessing example in the python-list:
# https://mail.python.org/pipermail/python-list/2009-February/538048.html

# BEGIN SPINNER_THREAD
import threading
import itertools
import time
import sys

class Signal:   # 定义一个简单的可变对象；其中有个 go 属性，用于从外部控制线程
    go = True

def spin(msg, singnal):  # 这个函数会在单独的线程中运行。signal 参数是 Signal 类的实例
    write, flush = sys.stdout.write, sys.stdout.flush
    for char in itertools.cycle('|/-\\'):  # 这其实是个无限循环，char 一直在几个字符间来回变
        status = char + ' ' + msg
        write(status)
        flush()
        write('\x08' * len(status))  # 使用退格符把光标移回来以显示动画
        time.sleep(1)
        if not singnal.go:  # 如果 go 的属性不是 True，就退出循环
            break
    write(' ' * len(status) + '\x08' * len(status))  # 使用空格清除状态信息，把光标移回开头


def slow_function():  # 假设这是耗时的计算
    # 假装等待 I/0 一段时间
    time.sleep(3)  # 调用 sleep 会阻塞主线程，不过一定要这么做，以便释放 GIL，创建从属线程
    return 42


def supervisor():  # 这个函数设置从属线程，显示线程对象，运行耗时的计算，最后杀死线程
    signal = Signal()
    spinner = threading.Thread(target=spin,
                               args=('thinking!', signal))
    print('spinner object:', spinner)  # 显示从属线程对象
    spinner.start()  # 启动从属线程
    result = slow_function()  # 运行 slow_function 函数，阻塞主线程；同时，从属线程以动画的形式旋转指针。
    signal.go = False  # 改变 signal 的状态；这会终止 spin 函数中的 for 循环
    spinner.join()  # 等待 spinner 线程结束
    return result


def main():
    result = supervisor()  # 运行 supervisor 函数
    print('Answer:', result)


if __name__ == '__main__':
    main()
# END SPINNER_THREAD


可惜这得在命令行去跑

下面来看如何使用 @asyncio.coroutine 装饰器替代线程，实现相同的行为

In [ ]:
# BEGIN SPINNER_ASYNCIO
import asyncio
import itertools
import sys


@asyncio.coroutine  # 打算交给 asyncio 处理的协程要使用 @asyncio.coroutine 装饰。这不是强制要求，但强烈建议这么做。原因在后面说明
def spin(msg):  # 这里不需要 关闭线程的 signal 函数
    write, flush = sys.stdout.write, sys.stdout.flush
    for char in itertools.cycle('|/-\\'):
        status = char + ' ' + msg
        write(status)
        flush()
        write('\x08' * len(status))
        try:
            yield from asyncio.sleep(.1)  # 使用 yield from asyncio.sleep(.1) 代替 time.sleep(.1)，这样的休眠不会阻塞事件循环
        except asyncio.CancelledError:  # 如果 spin 苏醒后抛出 asyncio.CancelledError 异常，其原因是发出了取消请求，因此退出循环
            break
    write(' ' * len(status) + '\x08' * len(status))


@asyncio.coroutine
def slow_function():  # 现在，slow_function 函数是协程，在用休眠假装进行 I/0 操作时，使用 yield from asyncio.sleep(3) 继续执行事件循环
    # pretend waiting a long time for I/O
    yield from asyncio.sleep(3)  # yield from asyncio.sleep(3) 表达式把控制权交给主循环，在休眠结束后回复这个协程
    return 42


@asyncio.coroutine
def supervisor(loop):  # 现在，supervisor 函数也是协程，因此可以使用 yield from 驱动 slow_function 函数。
    spinner = loop.create_task(spin('thinking!'))  # asyncio.async(...) 函数排定 spin 协程的运行时间，使用一个 Task 对象包装 spin 协程，并立即返回
    print('spinner object:', spinner)  # 显示 Task 对象

    # 驱动 slow_function 函数，结束后，获取返回值。同时，事件循环继续运行，因为 slow_function 函数最后使用 yield from 表达式把控制权交回了主循环
    result = yield from slow_function()  
    spinner.cancel()  # Task 对象可以取消，取消后会在协程当前暂停的 yield 处抛出 asyncio.CancelledError 异常，协程可用捕获这个异常，也可以延迟取消，甚至拒绝取消
    return result


def main():
    loop = asyncio.get_event_loop()  # 获取事件循环的引用
    result = loop.run_until_complete(supervisor(loop))  # 驱动 supervisor 协程，让它运行完毕；这个协程的返回值是这次调用的返回值
    loop.close()
    print('Answer:', result)


if __name__ == '__main__':
    main()
# END SPINNER_ASYNCIO


这两版中 supervisor 函数的比较:
* asyncio.task 查不到与 threading.Thread 对象等效。
* Task 对象用于驱动协程，Thread 对象用于调用可调用的对象
* Task 对象不由自己手动实例化，而是通过把协程传给 loop.create_task(...) 函数方法获取
* 获取的 Task 对象以及排定了运行时间；Thread 实例则必须调用 start 方法，明确告知让他运行
* 在线程版 supervisor 函数中，slow_function 是普通的函数，直接由线程调用。在异步版 supervisor 函数中，函数中，slow_function 函数是协程，由 await 驱动
* 没有 API 能从外部终止线程，因为线程随时可能被中断，导致系统处于无效状态。如果想终止任务，可以使用 Task.cancel() 实例方法，在协程内部抛出 CancelledError 异常。协程可以在 await 处捕获这个异常，处理中止请求。
* supervisor 协程必须在 main 函数中由 loop.run_until_complete 方法执行  

## 18.1.1 asyncio.Future: 故意不阻塞

asyncio.Future 与 concurrent.futures.Future 类的接口基本一致，不过实现方式不同，不可以互换。  
在 asyncio 包中，loop.create_task(...) 方法接收一个协程，排定它的运行时间，然后返回一个 asyncio.Task 实例，这与 Executor.submit(...) 方法创建 concurrent.futures.Future 实例是一个道理。

与 concurrent.futures.Future 类似，asyncio.Future 类也提供了 .done()、.add_down_callback(...) 和 .result() 等方法。前两个方法与之前相似，不过 .result() 方法差别很大。

asyncio.Future 类的 .result() 方法没有参数，因此不能指定超过时间。此外，如果调用 .result() 方法时 future 还没运行完毕，那么 .result() 方法不会阻塞去等待结果，而是抛出 asyncio.InvalidStateError 异常。

然而，获取 asyncio.Future 对象的结果通常使用 await，从中产出结果，使用 await 处理 future，等待 future 运行完毕这一步无需我们担心，而且不会阻塞事件循环，因为在 asyncio 中，await 的作用是把控制权还给事件循环。

## 18.1.2 从 future、任务和协程中产出

# 18.2 使用 asyncio 和 aiohttp 包下载

In [ ]:
"""Download flags of top 20 countries by population

asyncio + aiottp version

Sample run::

    $ python3 flags_asyncio.py
    EG VN IN TR RU ID US DE CN MX JP BD NG ET FR BR PH PK CD IR
    20 flags downloaded in 1.07s

"""
# BEGIN FLAGS_ASYNCIO
import asyncio

import aiohttp  # <1>

from flags import BASE_URL, save_flag, show, main  # <2>


@asyncio.coroutine  # <3>
def get_flag(cc):
    url = '{}/{cc}/{cc}.gif'.format(BASE_URL, cc=cc.lower())
    resp = yield from aiohttp.request('GET', url)  # 阻塞操作通过协程实现
    image = yield from resp.read()  # 读取响应内容也是一项单独的异步操作
    return image


@asyncio.coroutine
def download_one(cc):  # 它也是协程
    image = yield from get_flag(cc)  # 调用协程
    show(cc)
    save_flag(image, cc.lower() + '.gif')
    return cc


def download_many(cc_list):
    loop = asyncio.get_event_loop()  # 获取事件循环
    to_do = [download_one(cc) for cc in sorted(cc_list)]  # 构建协程对象列表
    wait_coro = asyncio.wait(to_do)  # 虽然函数名是 wait，但它不是阻塞型函数。wait 是一个协程，等传给它的所有协程运行完毕后结束
    res, _ = loop.run_until_complete(wait_coro)  # 执行事件循环，直到 wait_coro 运行结束；事件循环的过程中，脚本会在这里阻塞。
    loop.close() # 关闭事件循环

    return len(res)


if __name__ == '__main__':
    main(download_many)
# END FLAGS_ASYNCIO


asyncio.wait(...) 协程的参数是一个由 future 或协程构成的可迭代对象；wait 会分别把各个协程包装进一个 Task 对象。最终的结果是，wait 处理的所有对象都通过某种方式变成 Future 类的实例。wait 是协程函数，因此返回一个协程或生成器对象；wait_coro 变量中存储的正是这种对象。为了驱动协程，我们把协程传给 loop.run_until_complete(...) 方法。

loop.run_until_complete 方法的参数是一个 future 或协程。如果是协程，run_until_complete 方法与 wait 函数易于，把协程包装进一个 Task 对象中。协程、future 和 任务都能由 await 驱动，这正是 run_until_complete 方法对 wait 函数返回的 wait_core 对象所做的事。wait_core 运行结束后返回一个元组，第一个元素是一系列结束的 future，第二个元素是一些列未结束的 future。在上面的示例中，我们将它赋值给 _，将其忽略，但 wait 函数有两个关键字参数，如果设定了可能会返回未结束的 future，这两个参数是 timeout 和 return_when。

await foo 方法能防止阻塞，是因为当前协程暂停后，控制权回到事件循环手中，再去驱动其他协程；foo、future 或协程运行完毕后，把结果返回给暂停的协程，将其恢复。

对于 await:
* 我们编写的协程链条始终通过把最外层委派生成器传给 asyncio 包 API 中的某个函数 (如 loop.run_until_complete) 驱动  
  也就是说，使用 asyncio 包时，我们编写的代码不通过调用 next 或 .send 方法驱动协程 ———— 这一点由 asyncio 包实现的事件循环去做。
* 我们编写的协程链接最终通过 await 把制作委托给 asyncio 包中的某个协程函数或协程方法，或者其他库中实现高层协议的协程。
  也就是说，最内层的子生成器是库中真正执行 I/0 操作的函数，而不是我们自己编写的函数

# 18.3 避免阻塞型调用

有两种方法能避免阻塞型调用中止整个应用程序的进程:
* 在单独的线程中运行各个阻塞型操作
* 把每个阻塞型操作转换成非阻塞的异步调用

为了降低内存消耗，通常使用回调来实现异步调用。这是一种底层概念，类似于所有并发机制中最古老、最原始的 ———— 硬件终端。使用回调时，我们不等待响应，而是注册一个函数，在发生某件事时调用。这样，所有调用都是非阻塞的，回调简单，而且消耗低。

# 18.4 改进 asyncio 下载脚本

## 18.4.1 使用 asyncio.as_completed 函数

为了更新进度条，各个协程运行结束后要立即获取结果，所以需要使用 as_completed。

我们需要重写几个函数；重写后的函数可以供 concurrent.future 版重用，之所以要重写，是因为在使用 asyncio 包的程序中只有一个主线程，而在这个线程中不能有阻塞型调用，因为事件循环也正在这个线程中运行。

In [ ]:
# BEGIN FLAGS2_ASYNCIO_TOP
import asyncio
import collections
import contextlib

import aiohttp
from aiohttp import web
import tqdm

from flags2_common import main, HTTPStatus, Result, save_flag

# default set low to avoid errors from remote site, such as
# 503 - Service Temporarily Unavailable
DEFAULT_CONCUR_REQ = 5
MAX_CONCUR_REQ = 1000


class FetchError(Exception):  # 这个自定义异常用于包装其他 HTTP 或网络异常，并获取 countrry_code，以便报告错误
    def __init__(self, country_code):
        self.country_code = country_code


@asyncio.coroutine
def get_flag(base_url, cc): # get_flag协程有三种返回结果: 下载到的图像、HTTP404 时抛出 web.HTTPNotFound 异常、返回其他HTTP状态码时，抛出 aiohttp.HttpProcessingError
    url = '{}/{cc}/{cc}.gif'.format(base_url, cc=cc.lower())
    resp = yield from aiohttp.request('GET', url)
    with contextlib.closing(resp):
        if resp.status == 200:
            image = yield from resp.read()
            return image
        elif resp.status == 404:
            raise web.HTTPNotFound()
        else:
            raise aiohttp.HttpProcessingError(
                code=resp.status, message=resp.reason,
                headers=resp.headers)


@asyncio.coroutine
def download_one(cc, base_url, semaphore, verbose):  # semaphore 参数是 asyncio.Semaphore 类的实例。Semaphore 类是同步装置，用于限制并发请求数量
    try:
        with (yield from semaphore):  # yield from 把 semaphore 当作上下文管理器使用，防止阻塞整个系统: 如果 semaphore 计数器的值是所允许的最大值，只有这个协程会阻塞
            image = yield from get_flag(base_url, cc)  # 退出这个 with 语句后，semaphore 计数器的值会低见，解除阻塞可能在等待同一个 semaphore 对象的其他协程实例
    except web.HTTPNotFound:  # 
        status = HTTPStatus.not_found
        msg = 'not found'
    except Exception as exc:
        raise FetchError(cc) from exc  # 
    else:
        save_flag(image, cc.lower() + '.gif')  # 
        status = HTTPStatus.ok
        msg = 'OK'

    if verbose and msg:
        print(cc, msg)

    return Result(status, cc)
# END FLAGS2_ASYNCIO_TOP

Semaphore 对象维护着一个内部距市区，若在对象上调用 .acquire() 协程方法，计数器则递减；若在对象上调用 .release() 方法，计数器会递增。计数器的初始值在实例化 Semaphore 时设定。如 downloader_coro 函数中的这一行
```python 
    semaphore = asyncio.Semaphore(concur_req)
```
如果计数器大于 0，那么调用 .acquire() 方法不会阻塞；如果计数器为 0，那么 .acquire() 方法会阻塞调用这个方法的协程，直到其他协程在同一个 Semaphore 对象上调用 .release() 方法，让计数器递增，在例子中，我们没有使用这两个方法而是把 Semaphore 当作上下文管理器使用
```python 
    with(yield from semaphore):
        image = yield from get_flag(...)
```

这段代码保证，任何时候都不会有超过 concur_req 个 get_flag 协程启动

In [ ]:
# BEGIN FLAGS2_ASYNCIO_DOWNLOAD_MANY
@asyncio.coroutine
def downloader_coro(cc_list, base_url, verbose, concur_req):  # 这个协程的参数与 download_may 一样，但不能直接调用，它是协程函数
    counter = collections.Counter()
    semaphore = asyncio.Semaphore(concur_req)  # 创建一个 asyncio.Semaphore 实例
    to_do = [download_one(cc, base_url, semaphore, verbose)
             for cc in sorted(cc_list)]  # 多次调用 download_one 协程，创建一个协程对象列表

    to_do_iter = asyncio.as_completed(to_do)  # 获取一个迭代器，这个迭代器会在 future 运行结束后返回 future
    if not verbose:
        to_do_iter = tqdm.tqdm(to_do_iter, total=len(cc_list))  # 把迭代器传给 tqdm 显示进度条
    for future in to_do_iter:  # 迭代运行结束的 future
        try:
            res = yield from future  # 尝试获取结果/异常
        except FetchError as exc:  # 捕获异常
            country_code = exc.country_code  # 获取异常代码
            try:
                error_msg = exc.__cause__.args[0]  # 尝试从原来的异常 __cause__ 中获取信息
            except IndexError:
                error_msg = exc.__cause__.__class__.__name__  # 如果原来的异常中找不到错误信息，那么使用所链接异常的类名作为错误信息
            if verbose and error_msg:
                msg = '*** Error for {}: {}'
                print(msg.format(country_code, error_msg))
            status = HTTPStatus.error
        else:
            status = res.status

        counter[status] += 1  # 记录结果

    return counter  # 返回计数器


def download_many(cc_list, base_url, verbose, concur_req):
    loop = asyncio.get_event_loop()
    coro = downloader_coro(cc_list, base_url, verbose, concur_req)
    counts = loop.run_until_complete(coro)  # download_namy 函数只是实例化的 downloader_coro 协程，然后通过 run_until_complete 方法传给事件循环
    loop.close()  # 循环结束后关闭事件循环，返回 counts

    return counts


if __name__ == '__main__':
    main(download_many, DEFAULT_CONCUR_REQ, MAX_CONCUR_REQ)
# END FLAGS2_ASYNCIO_DOWNLOAD_MANY

##  18.4.2 使用 Executor 对象，防止阻塞事件循环

Python 社区往往会忽略一个试试 ———— 访问本地文件系统会阻塞，想当然的认为这种操作不会受网络访问的高延迟影响。与之相比 Node.js 程序员则始终谨记，所有文件系统函数都会阻塞，因为这些函数的签名中都指明了要有回调。

在我们的下载脚本中，阻塞型函数是 save_flag。在这个脚本的线程版中 save_flag 会阻塞 download_one 的线程，但阻塞的只是众多工作线程中的一个。阻塞型 I/0 调用在背后会释放 GIL，因此另一个线程可以继续，但在 asyncio 版，save_flag 函数阻塞了客户代码与 asyncio 事件循环共用的唯一线程，因此保存文件时，整个应用程序都会冻结。这个问题的解决方法是，使用事件循环对象的 run_in_executor 方法。

asyncio 的事件循环在背后维护者一个 ThreadPoolExecutor 对象，我们可以调用 run_in_executor 方法，把可调用的对象发给它指向。

In [ ]:
# BEGIN FLAGS2_ASYNCIO_EXECUTOR
async def download_one(cc, base_url, semaphore, verbose):
    try:
        with (await semaphore):
            image = await get_flag(base_url, cc)
    except web.HTTPNotFound:
        status = HTTPStatus.not_found
        msg = 'not found'
    except Exception as exc:
        raise FetchError(cc) from exc
    else:
        loop = asyncio.get_event_loop()  # 获取事件循环对象的引用
        loop.run_in_executor(None,  # run_in_executor 方法的第一个参数是 Executor 实例，如果设为 None，使用事件循环默认 ThreadPoolExecutor 实例
                save_flag, image, cc.lower() + '.gif')  # 余下的参数是可调用的对象，以及可调用对象的位置参数
        status = HTTPStatus.ok
        msg = 'OK'

    if verbose and msg:
        print(cc, msg)

    return Result(status, cc)
# END FLAGS2_ASYNCIO_EXECUTOR

# 18.5 从回调到 future 和协程

回调地狱：如果一个操作需要依赖之前操作的结果，那就得嵌套回调，如果要连续做三次异步调用，那就需要嵌套三层回调。

## 每次下载发起多次请求

假设保存每面国企时，我们不仅想在文件名中使用国家代码，还想加上国家名称。那么，下载每面国旗时要发起两个请求: 一个请求用于获取国旗，另一个请求用于获取图像所在目录的 metadata.json 文件(记录着国家名称)

在同一个任务中发起多个请求，这对线程版脚本来说很容易: 只需接连发起两次请求，阻塞线程两次。

我们可以修改脚本以实现:
* download_one : 现在，这个协程使用 yield from 把指责委托给 get_flag 协程和新添的 get_country 协程
* get_flag 这个协程的大多数代码移到新的 http_get 协程了，以便也能在 get_countey 协程中使用
* get_countey 获取国家代码响应的 metadata.json 文件，从中读取国家名称
* http_get 从 Web 获取文件的通用代码

In [ ]:
"""Download flags and names of countries.

asyncio version
"""

import asyncio
import collections

import aiohttp
from aiohttp import web
import tqdm

from flags2_common import main, HTTPStatus, Result, save_flag

# default set low to avoid errors from remote site, such as
# 503 - Service Temporarily Unavailable
DEFAULT_CONCUR_REQ = 5
MAX_CONCUR_REQ = 1000


class FetchError(Exception):
    def __init__(self, country_code):
        self.country_code = country_code

# BEGIN FLAGS3_ASYNCIO
@asyncio.coroutine
def http_get(url):
    res = yield from aiohttp.request('GET', url)
    if res.status == 200:
        ctype = res.headers.get('Content-type', '').lower()
        if 'json' in ctype or url.endswith('json'):
            data = yield from res.json()  # 如果内容类型包含 'json'，或者 url 以 .json 结尾，那么在响应上调用 .json() 方法，解析响应
        else:
            data = yield from res.read()  # 如果是国旗则用 .read() 方法读取原始字节
        return data

    elif res.status == 404:
        raise web.HTTPNotFound()
    else:
        raise aiohttp.errors.HttpProcessingError(
            code=res.status, message=res.reason,
            headers=res.headers)


@asyncio.coroutine
def get_country(base_url, cc):
    url = '{}/{cc}/metadata.json'.format(base_url, cc=cc.lower())
    metadata = yield from http_get(url)  # metadata 的变量值是一个由 JSON 内容构建的 Python 字典
    return metadata['country']


@asyncio.coroutine
def get_flag(base_url, cc):
    url = '{}/{cc}/{cc}.gif'.format(base_url, cc=cc.lower())
    return (yield from http_get(url)) # <4>


@asyncio.coroutine
def download_one(cc, base_url, semaphore, verbose):
    try:
        with (yield from semaphore): # 分别在 semaphore 控制的两个 with 块中调用 get_flag 和 get_country
            image = yield from get_flag(base_url, cc)
        with (yield from semaphore):
            country = yield from get_country(base_url, cc)
    except web.HTTPNotFound:
        status = HTTPStatus.not_found
        msg = 'not found'
    except Exception as exc:
        raise FetchError(cc) from exc
    else:
        country = country.replace(' ', '_')
        filename = '{}-{}.gif'.format(country, cc)
        loop = asyncio.get_event_loop()
        loop.run_in_executor(None, save_flag, image, filename)
        status = HTTPStatus.ok
        msg = 'OK'

    if verbose and msg:
        print(cc, msg)

    return Result(status, cc)
# END FLAGS3_ASYNCIO

# 18.6 使用 asyncio 包编写服务器

示例服务器的作用是，让客户端使用 unicodedata 模块，通过规范名称查找 Unicode 字符。

## 18.6.1 使用 asynico 包编写 TCP 服务器

In [ ]:
# BEGIN TCP_CHARFINDER_TOP
import sys
import asyncio

from charfinder import UnicodeNameIndex  # UnicodeNameIndex 类用于构建名称索引，提供查询方法

CRLF = b'\r\n'
PROMPT = b'?> '

index = UnicodeNameIndex()  # 实例化 UnicodeNameIndex 类时，他会使用 XXX 文件或构建这个文件。

@asyncio.coroutine
def handle_queries(reader, writer):  # 这个下次要传给 asyncio.start_server 函数，接收的两个参数是 asyncio.StreamReader 对象和 asyncio.StreamWrite 对象
    while True:  # 这个循环处理绘画，直到客户端收到控制字符退出
        writer.write(PROMPT)  # can't yield from!  # StreamWrite.write 方法不是协程，只是普通函数，它发送 ?> 提示符
        yield from writer.drain()  # must yield from!  # StreamWrite.drain 方法刷新 writer 缓冲，它是协程
        data = yield from reader.readline()  # StreamReader.readline 方法是协程，返回一个 bytes 对象
        try:
            query = data.decode().strip()
        except UnicodeDecodeError:  # Telent 客户端发送控制字符时，可能会抛出 UnicodeDecodeError 异常，遇到这种情况时，为了简单起见，假装发送的是空字符
            query = '\x00'
        client = writer.get_extra_info('peername')  # 返回与套接字链接的远程地址
        print('Received from {}: {!r}'.format(client, query))  # 在服务器控制台中记录查询
        if query:
            if ord(query[:1]) < 32:  # 如果收到控制字符或空字符，退出循环
                break
            lines = list(index.find_description_strs(query)) # 返回一个生成器，产出包含 Unicode 码位、真正的字符和字符名称的字符串
            if lines:

                # 使用默认的 UTF-8 编码把 lines 转换成 bytes 对象，并在每一行末尾添加回车符和换行符；注意，参数是一个生成器表达式
                writer.writelines(line.encode() + CRLF for line in lines) 
            writer.write(index.status(query, len(lines)).encode() + CRLF) # 输出状态

            yield from writer.drain()  # 刷新输出缓冲
            print('Sent {} results'.format(len(lines)))  # 控制台中记录响应

    print('Close the client socket')  # 控制台中记录会话结束
    writer.close()  # 关闭 StreamWriter 流
# END TCP_CHARFINDER_TOP

handle_queries 协程的名称是复数，因为它启动交互式会话后能处理各个客户端发来的多次请求

In [ ]:
# BEGIN TCP_CHARFINDER_MAIN
def main(address='127.0.0.1', port=2323):  # 调用 main 函数时可以不传入参数
    port = int(port)
    loop = asyncio.get_event_loop()
    server_coro = asyncio.start_server(handle_queries, address, port,
                                loop=loop) # asyncio.start_server 协程运行结束后，返回的协程对象返回一个 asyncio.Server 实例，即一个 TCP 套接字服务器
    server = loop.run_until_complete(server_coro) # 驱动 server_coro 协程，启动服务器

    host = server.sockets[0].getsockname()  # 获取这个服务器的第一个套接字的地址和端口
    print('Serving on {}. Hit CTRL-C to stop.'.format(host))  # 控在制台显示
    try:
        loop.run_forever()  # 运行事件循环，main 函数在这里阻塞，直到在服务器的控制台中按 CTRL-C 才会关闭
    except KeyboardInterrupt:  # CTRL+C pressed
        pass

    print('Server shutting down.')
    server.close()  # 关闭服务器
    loop.run_until_complete(server.wait_closed())  # server.wait_closed() 返回一个 future；调用 loop.run_until_compete 方法，运行 future
    loop.close()  # 终止事件循环


if __name__ == '__main__':
    main(*sys.argv[1:])  # 处理可选命令行参数的简便方式: 展开 sys.argv[1:] 传给 main 函数，未指定的参数使用相应的默认值
# END TCP_CHARFINDER_MAIN

## 18.6.2 使用 aiohttp 包编写 Web 服务器

In [ ]:
# BEGIN HTTP_CHARFINDER_SETUP
@asyncio.coroutine
def init(loop, address, port):  # init 协程产出一个服务器，交给事件循环驱动
    app = web.Application(loop=loop)  # asyncio.web.Application 类表示 Web 应用
    app.router.add_route('GET', '/', home)  # 通过路由把 URL 模式映射到处理函数上；这里把 GET /路由映射到 home 函数上
    handler = app.make_handler()  # app.make_handler 方法返回一个 aiohttp.web.RequestHandler 实例，根据 app 对象设置的路由处理 HTTP 请求
    server = yield from loop.create_server(handler,
                                           address, port)  # create_server 方法创建服务器，以 handler 为协议处理程序，并把服务器绑定在指定的地址上
    return server.sockets[0].getsockname()  # 返回第一个服务器套接字的地址和端口

def main(address="127.0.0.1", port=8888):
    port = int(port)
    loop = asyncio.get_event_loop()
    host = loop.run_until_complete(init(loop, address, port))  # 运行 init 函数，启动服务器，获取地址和接口
    print('Serving on {}. Hit CTRL-C to stop.'.format(host))
    try:
        loop.run_forever()  # 运行事件循环，控制权的事件循环手上时，main 函数会在这里阻塞
    except KeyboardInterrupt:  # CTRL+C pressed
        pass
    print('Server shutting down.')
    loop.close()  # 关闭事件循环


if __name__ == '__main__':
    main(*sys.argv[1:])
# END HTTP_CHARFINDER_SETUP

对比 TCP 和 HTTP 服务器的创建

asyncio.start_server 函数和 loop.creater_server 方法都是协程，返回的结果都是 asyncio.Server 对象。为了启动服务器并返回服务器的引用，这两个协程都要由它让驱动完成运行。

只有驱动协程，协程才能做事，而驱动协程有两个方法，要么使用 await，要么传给 asyncio 包中某个参数为协程或future的函数

In [ ]:
# BEGIN HTTP_CHARFINDER_HOME
def home(request):  # 一个路由处理函数，参数是一个 aiohttp.web.Request 实例
    query = request.GET.get('query', '').strip()  # 获取查询字符串，去掉首尾空白
    print('Query: {!r}'.format(query))  #在服务器的控制台中记录查询
    if query:  # 如果有查询字符串，从索引中找到结果后返回
        descriptions = list(index.find_descriptions(query))
        res = '\n'.join(ROW_TPL.format(**descr._asdict())
                        for descr in descriptions)
        msg = index.status(query, len(descriptions))
    else:
        descriptions = []
        res = ''
        msg = 'Enter words describing characters.'

    html = template.format(query=query, result=res,  # 渲染 HTML 界面
                           message=msg)
    print('Sending {} results'.format(len(descriptions)))  # 在控制台中记录响应
    return web.Response(content_type=CONTENT_TYPE, text=html) # 构建 Response 对象，将其返回
# END HTTP_CHARFINDER_HOME